In [17]:
import numpy as np

In [18]:
class Clause:
  def __init__(self,inputCount, maxStates, LearningRate,id):
    from random import random
    self.rand = random
    self.maxStates = maxStates
    self.LearningRate = LearningRate
    self.s = 1/LearningRate
    self.cutoff = (maxStates>>1)
    self.inputCount = (inputCount<<1)
    self.States = np.ones((inputCount<<1))*self.cutoff
    self.id = id
  def getOutput(self,inp): #inp is an array of literals
    return np.logical_and.reduce(np.where((self.States>self.cutoff),inp,True))
  def real1(self,inp): #inp is an array of features
    inp = (np.array([inp,(inp^True)]).T).reshape(inp.shape[0]*2) #conversion of inp from array of features to array of literals
    machine = np.logical_and.reduce(np.where((self.States>self.cutoff),inp,True)) #clause output for given input
    # real = 1
    if (machine and (self.rand()<=self.s)):
      self.States = np.minimum(self.maxStates,(self.States + (np.ones(self.inputCount)*inp)))
    elif (self.rand()>self.s):
      self.States = np.maximum(0,(self.States - 1))
  def real0(self,inp):
    inp = (np.array([inp,(inp^True)]).T).reshape(inp.shape[0]*2)
    machine = np.logical_and.reduce(np.where((self.States>self.cutoff),inp,True))
    if machine:
      self.States = np.minimum(self.maxStates,(self.States + 1))
  def __repr__(self):
    return repr(self.States)
  def __str__(self):
    return repr(self.States)

In [19]:
a = np.array([1,2,3,4])
b = np.array([False,False,False,False])
np.where(b,(a>2),True)

array([ True,  True,  True,  True])

In [20]:
class MNIST:
  def __init__(self,inputCount,maxStates, LearningRate):
    self.TM = []
    for id in range(1,11):
      self.TM.append(Clause(inputCount,maxStates,LearningRate,id))
  def Train(self,data):
    from random import randint
    for entry in data:
      real = int(entry[0])
      img = np.array(entry[1:])
      while True:
        conjugate = randint(1,10)
        if conjugate!=real:
          break
      self.TM[real-1].real1(img)
      self.TM[conjugate-1].real0(img)
  def getOutput(self,img):
    img = (np.array([img,(img^True)]).T).reshape(img.shape[0]*2)
    out = []
    for machine in self.TM:
      out.append(machine.getOutput(img))
    return out
  def getClassSum(self,img):
    out = np.array(self.getOutput(img))*np.ones(10)
    return out*(2*out - np.add.reduce(2*out - 1))
  def getPrintableOutput(self,img):
    out = (self.getClassSum(img))
    sum = np.add.reduce(np.exp(out))
    out = np.exp(out)/sum
    import matplotlib.pyplot as plt
    plt.bar(range(1,11),out*100)
    plt.xticks(range(1,11))
  def showMe(self):
    temp = []
    for i in self.TM:
      temp.append(i.States)
    return np.array(temp)
  def accuracy(self,data):
    totalCases, correctCases, partialCorrect = 0,0,0
    for entry in data:
      totalCases += 1
      machine = self.getOutput(entry[1:])
      if np.add.reduce(machine)>1:
        partialCorrect += 1
        continue
      if machine[entry[0]-1]==1:
        correctCases += 1
        partialCorrect += 1
    return ((correctCases*100)/totalCases), ((partialCorrect*100)/totalCases)

In [21]:
with open("sample_data/mnist_train_small.csv") as f:
  from csv import reader
  stream = reader(f)
  data = []
  for i in stream:
    data.append([int(i[0])]+[(int(a)>10) for a in i[1:]])
  data = np.array(data,copy=True)

In [3]:
with open("sample_data/mnist_test.csv") as f:
  from csv import reader
  stream = reader(f)
  dataTest = []
  for i in stream:
    dataTest.append([int(i[0])]+[(int(a)>10) for a in i[1:]])
  print(len(dataTest[0]))

785


In [23]:
model = MNIST((28*28),100,2.5)

In [24]:
for rerun in range(10):
  model.Train(data)
  c, p = model.accuracy(data)
  print(f"Completed Epoch-{rerun} with accuracy {c}% ({p}% partially)")

#Partial Means here that machine classified into more than one number
#Example: Machine: Clause-1 : 0
#Example: Machine: Clause-2 : 1
#Example: Machine: Clause-3 : 0
#Example: Machine: Clause-4 : 1
#Example: Machine: Clause-5 : 0
#Example: Machine: Clause-6 : 0
#Example: Machine: Clause-7 : 0
#Example: Machine: Clause-8 : 0
#Example: Machine: Clause-9 : 0
#Example: Machine: Clause-10: 0
#If Correct answer is 2 then machine is Incorrect but partially correct

Completed Epoch-0 with accuracy 0.27% (78.44% partially)
Completed Epoch-1 with accuracy 23.52% (71.515% partially)
Completed Epoch-2 with accuracy 23.16% (71.73% partially)
Completed Epoch-3 with accuracy 25.115% (71.735% partially)
Completed Epoch-4 with accuracy 22.89% (74.66% partially)
Completed Epoch-5 with accuracy 24.785% (73.86% partially)
Completed Epoch-6 with accuracy 24.215% (73.085% partially)
Completed Epoch-7 with accuracy 21.95% (75.64% partially)
Completed Epoch-8 with accuracy 24.09% (73.35% partially)
Completed Epoch-9 with accuracy 23.63% (74.375% partially)


In [25]:
#On the Test Set
model.accuracy(dataTest)

(23.63, 74.375)